In [1]:
import minari
import os
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
import shimmy
from custom_dmc_tasks import point_mass_maze

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise

from torch.utils.data import Dataset, DataLoader
# os.environ['MUJOCO_GL'] = 'glfw'

In [2]:
task = 'reach_bottom_right'

In [3]:
def render_environment():
    raw_env = point_mass_maze.make(task=task)
    raw_env.reset()
    
    physics = raw_env.physics
    frame = physics.render(
            height=480, 
            width=480, 
            camera_id=0, 
            )

    plt.figure(figsize=(8, 8))
    plt.imshow(frame)
    plt.axis('off')
    plt.title("PointMass Maze")
    plt.show()


In [4]:

from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

def make_env():
    raw_env = point_mass_maze.make(task=task)
    return shimmy.DmControlCompatibilityV0(raw_env)

# Замените SubprocVecEnv на DummyVecEnv
env = DummyVecEnv([make_env for _ in range(4)]) # Начните с 4, чтобы не забить память
env = VecMonitor(env)

policy_kwargs = dict(
    net_arch=dict(
        pi=[512, 512, 512],  # (Actor/Policy)
        qf=[512, 512, 512]   # (Critic/Q-function)
    )
)

model = TD3(
    "MultiInputPolicy",
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=1e-4,      # КРИТИЧНО: уменьшаем по Таблице 8
    buffer_size=1_000_000,
    batch_size=256,         # По таблице (Common -> Batch Size)
    tau=0.005,                # Эквивалент EMA 0.99 (1 - 0.99)
    gamma=0.99,              # По таблице (Discount Factor)
    target_policy_noise=0.2, 
    target_noise_clip=0.3,
    policy_delay=2,          # Policy Update Frequency = 1
    verbose=1,
    device='cpu'
)


Using cpu device


In [ ]:
model.learn(total_timesteps=1500_000, progress_bar=True)


In [ ]:
model.save(f"td3_point_mass_expert_{task}")

In [7]:
import imageio
import numpy as np

def create_agent_gif(model_path="td3_point_mass_expert", gif_name="agent_trajectory.gif"):
    # 1. Инициализация (используем raw_env для доступа к физике)
    raw_env = point_mass_maze.make(task=task)
    env = shimmy.DmControlCompatibilityV0(raw_env) # Убираем render_mode отсюда
    
    model = TD3.load(model_path)
    
    obs, _ = env.reset()
    frames = []
    
    print("Запуск агента...")
    # 500 шагов — это 10 секунд при 0.02s шаге. 5000 — это слишком много для GIF.
    for _ in range(500): 
        # Рендерим напрямую из физики dm_control (самый надежный способ)
        frame = raw_env.physics.render(height=480, width=480, camera_id=0)
        frames.append(frame)
        
        action, _ = model.predict(obs, deterministic=True)
        obs, _, terminated, truncated, _ = env.step(action)
        
        if terminated or truncated:
            break

    if frames:
        # Важно: приводим к типу uint8 перед сохранением
        imageio.mimsave(gif_name, [np.array(f).astype(np.uint8) for f in frames], 
                        fps=30) # Используем fps вместо duration для предсказуемости
        print(f"GIF успешно сохранен: {gif_name}")
    else:
        print("Ошибка: кадры не были сгенерированы!")

if __name__ == "__main__":
    # Убедись, что переменная task определена (например, task = 'point_mass_maze')
    create_agent_gif(model_path=f"td3_point_mass_expert_{task}", 
                     gif_name=f"agent_trajectory_{task}.gif")

Запуск агента...

/Users/sgerasimov/Desktop/GenAI/TDFlow/.venv/lib/python3.11/site-packages/imageio/plugins/pillow.py:410: 
DeprecationWarning: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == 
`duration=20` (1000 * 1/50).
  warnings.warn(

GIF успешно сохранен: agent_trajectory_reach_bottom_right.gif